In [1]:
import sys
import os
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.linear_model import Ridge
from scipy.optimize import minimize
import warnings

warnings.filterwarnings('ignore')


In [2]:
sys.path.append(os.path.abspath("/home/bk_anupam/code/ML/ML_UTILS/"))

In [3]:
import train_tabular_utils as tt
import cv_split_utils
import enums
from enums import ModelName
import data_utils

In [4]:
class Config:
    RUN_MODE = "LOCAL"
    RANDOM_SEED = 42
    NUM_FOLDS = 5
    MODEL_TYPE = ModelName.L2_Ridge
    TARGET_COL_NAME = "price"            
    METRIC = enums.Metrics.RMSE
    TRAIN_SINGLE_FOLD = False     

COLS_TO_LEAVE = ["id", "price", "kfold", "transmission_speed", "target_grp"]
CPU_COUNT = os.cpu_count()

DATA_READPATH = "./data/"
BASE_MODELS_PATH = "./output/"

In [5]:
# key is model type and value is list of experiment serial number of trained models for that type to be used in ensemble
models = {
    ModelName.CatBoost: [8, 9],    
    ModelName.LGBM: [5, 6, 7],
    ModelName.NeuralNet: [12]    
}

# for each key in models dictionary iterate through the values list using for comprehension
base_model_names = [f"{key}{i}" for key, value in models.items() for i in value]
base_model_names


['CatBoost8', 'CatBoost9', 'LightGBM5', 'LightGBM6', 'LightGBM7', 'NN12']

In [6]:
df_submission = pd.read_csv(DATA_READPATH + "sample_submission.csv")
df_oof_preds = pd.DataFrame()
df_test_preds = pd.DataFrame()
# load the OOF csv for each model
for model_name in base_model_names:
    df_model_oof = pd.read_csv(f"{BASE_MODELS_PATH}df_val_preds_{model_name}.csv")
    df_model_test_preds = pd.read_csv(f"{BASE_MODELS_PATH}df_test_preds_{model_name}.csv")    
    df_oof_preds[f"{model_name}_preds"] = df_model_oof["oof_preds"]
    df_test_preds[f"{model_name}_preds"] = df_model_test_preds["test_preds"]
df_oof_preds[Config.TARGET_COL_NAME] = df_model_oof[Config.TARGET_COL_NAME]

FileNotFoundError: [Errno 2] No such file or directory: './output/df_test_preds_NN12.csv'

In [17]:
df_oof_preds.head(1)

,CatBoost8_preds,CatBoost9_preds,LightGBM5_preds,LightGBM6_preds,LightGBM7_preds,price
0,40288.357971,19667.143514,40203.523906,40921.364539,37383.47234,488880.0


In [ ]:
def rmse_func(weights, oof_preds, target):
    pred = (oof_preds * weights).sum(axis=1)
    rmse = np.sqrt(1 / len(pred) * ((target - pred)**2).sum())
    return rmse

In [ ]:
# Start by giving equal weight to each model
n_models = len(base_model_names)
initial_weights = np.ones(n_models) / n_models
initial_weights